In [1]:
import os

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_KEY"] = ""
os.environ["AZURE_OPENAI_ENDPOINT"] = ""
os.environ["OPENAI_API_VERSION"] = "2023-05-15"

In [2]:
# # account for deprecation of LLM model
# import datetime
# # Get the current date
# current_date = datetime.datetime.now().date()

# # Define the date after which the model should be set to "gpt-3.5-turbo"
# target_date = datetime.date(2024, 6, 12)

# # Set the model variable based on the current date
# if current_date > target_date:
#     llm_model = "gpt-3.5-turbo"
# else:
#     llm_model = "gpt-3.5-turbo-0301"

In [3]:
customer_email = """
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,
the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
"""

In [4]:
style = """American English in a calm and respectful tone
"""

In [5]:
prompt = f"""Translate the text that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,
the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [6]:
# from langchain.chat_models import AzureChatOpenAI
from langchain_openai import AzureChatOpenAI

### Model

In [33]:
chat = AzureChatOpenAI(temperature=0, azure_deployment = "gpt4", verbose=True)
chat

AzureChatOpenAI(verbose=True, client=<openai.resources.chat.completions.Completions object at 0x000002688A264460>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002688A0D5970>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='', azure_endpoint='https://corestackgenai.openai.azure.com', deployment_name='gpt4', openai_api_version='2023-05-15', openai_api_type='azure')

### Prompt template

In [22]:
template_string = """
Translate the text that is delimited by triple backticks
Into a style that is {style}
{text}
"""

In [23]:
from langchain_core.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(template_string)

In [24]:
prompt_template.messages[0]

PromptTemplate(input_variables=['style', 'text'], template='\nTranslate the text that is delimited by triple backticks\nInto a style that is {style}\n{text}\n')

In [25]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [26]:
customer_style = """American English in a calm and respectful tone
"""

In [27]:
customer_email = """
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, 
the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
"""

In [28]:
customer_messages = prompt_template.format_messages(
                    style = customer_style,
                    text = customer_email
                    )

In [29]:
print(type(customer_messages))
print(customer_messages[0])

<class 'list'>
content="\nTranslate the text that is delimited by triple backticks\nInto a style that is American English in a calm and respectful tone\n\n\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, \nthe warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n\n"


In [30]:
customer_response = chat.invoke(customer_messages)
print(customer_response.content)

I'm quite upset that my blender lid came off and splattered my kitchen walls with smoothie! To add to the frustration, the warranty doesn't cover the cost of cleaning up my kitchen. I need your help immediately, please!


In [21]:
service_reply = """Hey there customer, the warranty does not cover cleaning expenses for your kitchen 
because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. 
Tough luck! See ya!
"""

In [22]:
service_style_pirate = """a polite tone that speaks in English Pirate"""

In [23]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

In [24]:
service_response = chat.invoke(service_messages)
print(service_response.content)

Ahoy there, me hearty, the warranty be not extendin' to cover cleanin' costs for yer galley, on account of it bein' yer own blunder that ye mishandled yer mixin' contraption by forgettin' to secure the top afore settin' it in motion. Hard cheese, matey! Until we cross paths again!


### Parser

In [25]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [26]:
customer_review = """
This leaf blower is pretty amazing.  It has four settings: candle blower, gentle breeze, windy city, and tornado. 
It arrived in two days, just in time for my wife's anniversary present. 
I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been 
using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers 
out there, but I think it's worth it for the extra features.
"""

review_template = """For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [27]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'))]


In [28]:
messages = prompt_template.format_messages(text=customer_review)

In [29]:
chat = AzureChatOpenAI(model_name="gpt-4", temperature=0, deployment_name="gpt4", verbose=True)
service_response = chat.invoke(messages)
print(service_response.content)

{"gift": true, "delivery_days": 2, "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}


In [30]:
type(service_response.content)

str

In [31]:
print(service_response.content.get("gift"))

AttributeError: 'str' object has no attribute 'get'

In [37]:
from langchain.output_parsers import StructuredOutputParser
from langchain.output_parsers import ResponseSchema

In [38]:
gift_schema = ResponseSchema(name="gift",
                            description="Was the item purchased as a gift for someone else? Answer True if yes, \
                             False if not or unknown.")

delivery_days_schema = ResponseSchema(name="delivery_days",
                                     description="How many days did it take for the product to arrive? If this information \
                                      is not found, output -1")

price_value_schema = ResponseSchema(name="price_value",
                                   description="Extract any sentences about the value or price,and output them as \
                                    a comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [39]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

In [40]:
review_template_2 = """
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [41]:
print(messages[0].content)


For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: 
This leaf blower is pretty amazing.  It has four settings: candle blower, gentle breeze, windy city, and tornado. 
It arrived in two days, just in time for my wife's anniversary present. 
I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been 
using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers 
out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" 

In [42]:
response = chat.invoke(messages)

In [43]:
print(response.content)

```json
{
	"gift": "True",
	"delivery_days": "2",
	"price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```


In [44]:
output_dict = output_parser.parse(response.content)

In [45]:
output_dict.get('delivery_days')

'2'

In [46]:
type(output_dict)

dict

## Memory

In [47]:
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

### ConversationBufferMemory

In [48]:
chat = AzureChatOpenAI(model_name="gpt-4", temperature=0, deployment_name="gpt4", verbose=True)
memory = ConversationBufferMemory()
conversation = ConversationChain(
                llm= chat,
                memory= memory,
                verbose= True
                )

In [49]:
conversation.predict(input= "Hii my name is imran")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hii my name is imran
AI:

> Finished chain.


"Hello Imran! It's nice to meet you. How can I assist you today?"

In [50]:
conversation.predict(input= "what is cosine similarity")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hii my name is imran
AI: Hello Imran! It's nice to meet you. How can I assist you today?
Human: what is cosine similarity
AI:

> Finished chain.


"Cosine similarity is a measure used to determine how similar two vectors are. It's often used in natural language processing and information retrieval to measure the similarity between two documents or sentences. \n\nThe cosine similarity is calculated by taking the dot product of two vectors and dividing it by the product of the two vectors' lengths (or magnitudes). The resulting value will be between -1 and 1, with 1 indicating that the vectors are identical, 0 indicating that the vectors are orthogonal (or not related), and -1 indicating that the vectors are diametrically opposed.\n\nIn the context of text analysis, each vector could represent a document or sentence, with the value of each element in the vector representing the frequency of a particular word. By comparing these vectors using cosine similarity, we can quantify how similar the documents or sentences are in terms of their word usage."

In [51]:
conversation.predict(input= "what is my name")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hii my name is imran
AI: Hello Imran! It's nice to meet you. How can I assist you today?
Human: what is cosine similarity
AI: Cosine similarity is a measure used to determine how similar two vectors are. It's often used in natural language processing and information retrieval to measure the similarity between two documents or sentences. 

The cosine similarity is calculated by taking the dot product of two vectors and dividing it by the product of the two vectors' lengths (or magnitudes). The resulting value will be between -1 and 1, with 1 indicating that the vectors are identical, 0 indicating that the vectors are orthogonal (or not related), and -1 indicating

'Your name is Imran, as you introduced yourself earlier in our conversation.'

In [52]:
memory.buffer

"Human: Hii my name is imran\nAI: Hello Imran! It's nice to meet you. How can I assist you today?\nHuman: what is cosine similarity\nAI: Cosine similarity is a measure used to determine how similar two vectors are. It's often used in natural language processing and information retrieval to measure the similarity between two documents or sentences. \n\nThe cosine similarity is calculated by taking the dot product of two vectors and dividing it by the product of the two vectors' lengths (or magnitudes). The resulting value will be between -1 and 1, with 1 indicating that the vectors are identical, 0 indicating that the vectors are orthogonal (or not related), and -1 indicating that the vectors are diametrically opposed.\n\nIn the context of text analysis, each vector could represent a document or sentence, with the value of each element in the vector representing the frequency of a particular word. By comparing these vectors using cosine similarity, we can quantify how similar the docume

In [53]:
memory.load_memory_variables({})

{'history': "Human: Hii my name is imran\nAI: Hello Imran! It's nice to meet you. How can I assist you today?\nHuman: what is cosine similarity\nAI: Cosine similarity is a measure used to determine how similar two vectors are. It's often used in natural language processing and information retrieval to measure the similarity between two documents or sentences. \n\nThe cosine similarity is calculated by taking the dot product of two vectors and dividing it by the product of the two vectors' lengths (or magnitudes). The resulting value will be between -1 and 1, with 1 indicating that the vectors are identical, 0 indicating that the vectors are orthogonal (or not related), and -1 indicating that the vectors are diametrically opposed.\n\nIn the context of text analysis, each vector could represent a document or sentence, with the value of each element in the vector representing the frequency of a particular word. By comparing these vectors using cosine similarity, we can quantify how simila

In [54]:
memory = ConversationBufferMemory()
memory.save_context({"input":"hi"}, {"output":"whats up"})

In [55]:
print(memory.buffer)

Human: hi
AI: whats up


In [56]:
memory.load_memory_variables({})

{'history': 'Human: hi\nAI: whats up'}

In [57]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [58]:
print(memory.buffer)

Human: hi
AI: whats up
Human: Not much, just hanging
AI: Cool


### ConversationBufferWindowMemory

In [59]:
from langchain.memory import ConversationBufferWindowMemory

In [60]:
wmemory = ConversationBufferWindowMemory(k=1)   # 1 means remember only one conversation. if 2 then remember 2 conversation

In [61]:
wmemory.save_context({"input": "Hi"},
                    {"output": "What's up"})
wmemory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

In [62]:
print(wmemory.buffer)

Human: Not much, just hanging
AI: Cool


In [63]:
wmemory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [64]:
chat = AzureChatOpenAI(model_name="gpt-4", temperature=0, deployment_name="gpt4", verbose=True)
w1memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
                llm= chat,
                memory= w1memory,
                verbose= True
                )

In [65]:
conversation.predict(input="Hi, my name is Andrew")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


"Hello Andrew! It's nice to meet you. How can I assist you today?"

In [66]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'The sum of 1+1 is 2.'

In [67]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is 1+1?
AI: The sum of 1+1 is 2.
Human: What is my name?
AI:

> Finished chain.


"I'm sorry, but as an AI, I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality."

### ConversationTokenBufferMemory

In [68]:
from langchain.memory import ConversationTokenBufferMemory

In [69]:
chat = AzureChatOpenAI(model_name="gpt-4", temperature=0, deployment_name="gpt4", verbose=True)
tmemory = ConversationTokenBufferMemory(llm= chat, max_token_limit=50)

In [70]:
tmemory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
tmemory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
tmemory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [71]:
tmemory.load_memory_variables({})

{'history': 'Human: AI is what?!\nAI: Amazing!\nHuman: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

### ConversationSummaryBufferMemory

In [72]:
from langchain.memory import ConversationSummaryBufferMemory

In [73]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

smemory = ConversationSummaryBufferMemory(llm= chat, max_token_limit=100)
smemory.save_context({"input": "Hello"}, {"output": "What's up"})
smemory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
smemory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [74]:
conversation = ConversationChain(
    llm= chat, 
    memory = smemory,
    verbose=True
)

In [75]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI exchange greetings before the human asks about their schedule for the day. The AI informs them of a morning meeting with the product team, time set aside for working on the LangChain project, and a lunch meeting at an Italian restaurant with a customer interested in AI. The AI also reminds the human to bring their laptop to show the latest LLM demo.
Human: What would be a good demo to show?
AI:

> Finished chain.


'A good demo to show would be the latest version of our Language Learning Model (LLM). This model has been updated with new features such as real-time translation, personalized language learning paths, and an interactive interface. It would be beneficial to demonstrate how these features work in real-time. Additionally, you could show how the model adapts to different learning styles and paces, which could be a key selling point for our customer.'

In [76]:
smemory.load_memory_variables({})

{'history': 'System: The human and AI exchange greetings before the human asks about their schedule for the day. The AI informs them of a morning meeting with the product team, time set aside for working on the LangChain project, and a lunch meeting at an Italian restaurant with a customer interested in AI. The AI also reminds the human to bring their laptop to show the latest LLM demo. The human then asks the AI for suggestions on a good demo to show.\nAI: A good demo to show would be the latest version of our Language Learning Model (LLM). This model has been updated with new features such as real-time translation, personalized language learning paths, and an interactive interface. It would be beneficial to demonstrate how these features work in real-time. Additionally, you could show how the model adapts to different learning styles and paces, which could be a key selling point for our customer.'}

## Chains

In [77]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [78]:
chat = AzureChatOpenAI(model_name="gpt-4", temperature=0, deployment_name="gpt4", verbose=True)

In [79]:
import pandas as pd
df = pd.read_csv("langchain_chain_data.csv")

In [80]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


In [81]:
prompt = ChatPromptTemplate.from_template("What is the best name to describe a company that makes {product}?")

In [82]:
chain = LLMChain(llm= chat, prompt= prompt)

In [83]:
product = "Queen Size Sheet Set"
chain.invoke(product)

{'product': 'Queen Size Sheet Set', 'text': '"Queen Comfort Bedding Co."'}

### SimpleSequentialChain

In [84]:
from langchain.chains import SimpleSequentialChain

In [85]:
chat = AzureChatOpenAI(model_name="gpt-4", temperature=0.9, deployment_name="gpt4", verbose=True)

In [86]:
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}?")
# Chain 1
chain_one = LLMChain(llm= chat, prompt=first_prompt)




second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following company:{company_name}")
# chain 2
chain_two = LLMChain(llm= chat, prompt=second_prompt)

In [87]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose= True)

In [88]:
overall_simple_chain.invoke(product)



> Entering new SimpleSequentialChain chain...
"Regal Linens"
Regal Linens offers a luxurious range of high-quality bedding, table linens, and towels, enhancing comfort with elegant designs.

> Finished chain.


{'input': 'Queen Size Sheet Set',
 'output': 'Regal Linens offers a luxurious range of high-quality bedding, table linens, and towels, enhancing comfort with elegant designs.'}

### SequentialChain

In [89]:
from langchain.chains import SequentialChain

In [90]:
chat = AzureChatOpenAI(model_name="gpt-4", temperature=0.9, deployment_name="gpt4", verbose=True)

In [91]:
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:""\n\n{Review}")

# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm= chat, prompt=first_prompt, output_key="English_Review")


In [92]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:" "\n\n{English_Review}")

# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm= chat, prompt=second_prompt, output_key="summary")


In [93]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template("What language is the following review:\n\n{Review}")

# chain 3: input= Review and output= language
chain_three = LLMChain(llm= chat, prompt=third_prompt, output_key="language")


In [94]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
"summary in the specified language:""\n\nSummary: {summary}\n\nLanguage: {language}")

# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm= chat, prompt=fourth_prompt, output_key="followup_message")


In [95]:
overall_chain = SequentialChain(chains=[chain_one, chain_two, chain_three, chain_four],
                               input_variables=["Review"],
                               output_variables=["English_Review", "summary","followup_message"],
                               verbose= True
                               )

In [96]:
review = df.Review[5]
overall_chain(review)

C:\Users\Imaran Asalam\AppData\Roaming\Python\Python39\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better... \nOld batch or counterfeit !?",
 'summary': 'The reviewer finds the taste average and questions the authenticity of the product as it differs in taste and quality from the same product purchased in stores.',
 'followup_message': "Réponse: Merci pour votre retour. Nous sommes désolés d'apprendre que vous avez trouvé le goût moyen et que vous questionnez l'authenticité de notre produit. Nous vous assurons que tous nos produits sont authentiques et de la plus haute qualité. Les variations de goût peuvent être dues à de nombreux facteurs, comme la durée de conservation. Nous apprécions vos commentaires et nous nous efforcerons d'améliorer l'expérience de nos produits à l'av

### RouterChain

In [97]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, 
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by describing the solution in imperative steps \
that a machine can easily interpret and you know how to choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [98]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [106]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [107]:
chat = AzureChatOpenAI(model_name="gpt-4", temperature=0.9, deployment_name="gpt4", verbose=True)

In [108]:

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm= chat, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [109]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm= chat, prompt=default_prompt)

In [110]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [113]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(chat, router_prompt)

In [114]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [115]:
chain.run("What is black body radiation?")

C:\Users\Imaran Asalam\AppData\Roaming\Python\Python39\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
C:\Users\Imaran Asalam\AppData\Roaming\Python\Python39\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(




> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation is a concept in physics that refers to the type of electromagnetic radiation that is emitted from an object that absorbs all radiation that falls on it. This is when the object is in thermal equilibrium with its environment. The radiation has a specific spectrum and intensity that depends only on the temperature of the body, not on its shape or composition. This theoretical or model body which emits radiation with the maximum possible intensity per unit of spectral frequency is known as a black body. \n\nThe concept of black body radiation is significant due to its role in the development of quantum mechanics. Physicist Max Planck studied black body radiation and, in the process, developed Planck's law which accurately describes the observed spectral radiance of black body radiation and it led him to propose the idea that energy is quantized. This was the beginning of quantum theory."

In [116]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...


C:\Users\Imaran Asalam\AppData\Roaming\Python\Python39\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [117]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...


C:\Users\Imaran Asalam\AppData\Roaming\Python\Python39\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


None: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


"Every cell in our body contains DNA because it is the blueprint or instruction manual for how to build and maintain that individual's body. It contains all the information needed for the growth, development and functioning of the body. It determines our physical characteristics, and even some of our behavior traits. The DNA in each cell is used to produce proteins, which are necessary for all bodily processes, from muscle contraction to digestion. DNA also allows for replication and repair of cells, which is crucial for growth and healing."

## Q&A Over Documents

In [17]:
chat = AzureChatOpenAI(model_name="gpt-35-turbo-instruct", temperature=0.9, deployment_name="gpt35_turbo_instruct", verbose=True)

In [18]:
chat

AzureChatOpenAI(verbose=True, client=<openai.resources.chat.completions.Completions object at 0x0000025BAD40C0A0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000025BAD4455B0>, model_name='gpt-35-turbo-instruct', temperature=0.9, openai_api_key='b739be5d1cbc4f87a6ca43cf1b0c5d14', openai_proxy='', azure_endpoint='https://corestackgenai.openai.azure.com', deployment_name='gpt35_turbo_instruct', openai_api_version='2023-05-15', openai_api_type='azure')

In [9]:
from langchain.chains import RetrievalQA
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores.docarray import DocArrayInMemorySearch
from langchain.indexes import VectorstoreIndexCreator

In [10]:
from langchain_openai import AzureOpenAIEmbeddings
embeddings = AzureOpenAIEmbeddings(azure_deployment = "ada002")

In [11]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path = file, encoding = 'utf-8')

In [5]:
index = VectorstoreIndexCreator(vectorstore_cls = DocArrayInMemorySearch, embedding = embeddings
                                ).from_loaders([loader])

In [6]:
query = "Please list all your shirts with sun protection in a table in markdown and summarize each one."

In [7]:
response = index.query(query)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: b739be5d********************5d14. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [13]:
from langchain.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(file_path=file, encoding='utf-8')

In [14]:
docs = loader.load()

In [15]:
docs[0]

Document(page_content="no: 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.", metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0})

In [ ]:
embed = embeddings.embed_query("Hi my name is Harrison")

In [16]:
print(len(embed))

In [17]:
print(embed[:5])

1536


In [18]:
db = DocArrayInMemorySearch.from_documents(docs, embeddings)

[-0.02186359354958571, 0.006734037540552159, -0.018200781829872538, -0.03919587420756086, -0.01404707648325098]


In [19]:
query = "Please suggest a shirt with sunblocking"

In [20]:
docs = db.similarity_search(query = query)

In [21]:
len(docs)

In [22]:
docs[0]

4

In [24]:
retriever = db.as_retriever()

In [35]:
from langchain_openai import AzureChatOpenAI
chat = AzureChatOpenAI(temperature=0, azure_deployment = "gpt4", verbose=True)
chat

AzureChatOpenAI(verbose=True, client=<openai.resources.chat.completions.Completions object at 0x000002688A223AC0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002688A227FD0>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='', azure_endpoint='https://corestackgenai.openai.azure.com', deployment_name='gpt4', openai_api_version='2023-05-15', openai_api_type='azure')

In [36]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])

In [39]:
response = chat.invoke(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") 


In [43]:
from IPython.display import display
from IPython.core.display import Markdown
display(Markdown(response.content))

| No. | Name | Description | Size & Fit | Fabric & Care | Additional Features | Sun Protection |
| --- | --- | --- | --- | --- | --- | --- |
| 255 | Sun Shield Shirt | High-performance sun shirt that protects from harmful UV rays. | Slightly Fitted: Softly shapes the body. Falls at hip. | 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated. Handwash, line dry. | Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant. | SPF 50+ sun protection, blocking 98% of the sun's harmful rays. |
| 374 | Men's Plaid Tropic Shirt, Short-Sleeve | Ultracomfortable sun protection shirt designed for fishing and extended travel. | N/A | 52% polyester and 48% nylon. UPF 50+ rated. Machine washable and dryable. | Front and back cape venting, two front bellows pockets. | UPF 50+ coverage, blocking 98% of the sun's harmful UV rays. |
| 618 | Men's Tropical Plaid Short-Sleeve Shirt | Lightest hot-weather shirt with superior protection from the sun's UV rays. | Traditional fit: Relaxed through the chest, sleeve, and waist. | 100% polyester. UPF 50+ rated. Wrinkle-resistant. | Front and back cape venting, two front bellows pockets. | SPF 50+ sun protection, blocking 98% of the sun's harmful rays. |
| 535 | Men's TropicVibe Shirt, Short-Sleeve | Sun-protection shirt with built-in UPF 50+ for hot and strong UV rays. | Traditional Fit: Relaxed through the chest, sleeve and waist. | Shell: 71% Nylon, 29% Polyester. Lining: 100% Polyester knit mesh. UPF 50+ rated. Machine wash and dry. | Wrinkle resistant. Front and back cape venting, two front bellows pockets. | SPF 50+ sun protection, blocking 98% of the sun's harmful rays. |

All four shirts offer high sun protection with a rating of UPF 50+, blocking 98% of the sun's harmful rays. They are designed for comfort and functionality, with features like moisture-wicking, abrasion resistance, wrinkle resistance, and venting. The fabrics used range from nylon and Lycra to polyester, ensuring durability and comfort.

In [44]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=chat, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [48]:
query2 =  "Please list all your shirts with sun protection in a table in markdown and summarize each one."

In [50]:
response = qa_stuff.run(query2)



> Entering new RetrievalQA chain...

> Finished chain.


In [51]:
display(Markdown(response))

Sure, here are the shirts with sun protection:

| No. | Name | Description |
| --- | --- | --- |
| 618 | Men's Tropical Plaid Short-Sleeve Shirt | This shirt is made of 100% polyester and is wrinkle-resistant. It has a traditional fit and is rated UPF 50+ for superior protection from the sun's UV rays. It also features front and back cape venting and two front bellows pockets. |
| 374 | Men's Plaid Tropic Shirt, Short-Sleeve | This shirt is made with 52% polyester and 48% nylon. It is designed for hot weather and offers UPF 50+ coverage. It is wrinkle-free and quickly evaporates perspiration. It also features front and back cape venting and two front bellows pockets. |
| 535 | Men's TropicVibe Shirt, Short-Sleeve | This shirt is made with 71% Nylon and 29% Polyester. It has a traditional fit and offers UPF 50+ sun protection. It is wrinkle resistant and features front and back cape venting and two front bellows pockets. |
| 255 | Sun Shield Shirt | This shirt is made with 78% nylon and 22% Lycra Xtra Life fiber. It is slightly fitted and offers UPF 50+ sun protection. It wicks moisture for quick-drying comfort and is abrasion resistant. It is recommended by The Skin Cancer Foundation as an effective UV protectant. |

## Create our own application

In [59]:
from langchain.chains import RetrievalQA
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores.docarray import DocArrayInMemorySearch
from langchain.indexes import VectorstoreIndexCreator
from langchain_openai.embeddings import AzureOpenAIEmbeddings
from langchain_openai.chat_models import AzureChatOpenAI


In [60]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
data = loader.load()

In [61]:
embeddings = AzureOpenAIEmbeddings(azure_deployment = "ada002")
embeddings

AzureOpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x00000268960E17C0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x0000026894321490>, model='text-embedding-ada-002', dimensions=None, deployment='ada002', openai_api_version='2023-05-15', openai_api_base=None, openai_api_type='azure', openai_proxy='', embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=set(), disallowed_special='all', chunk_size=16, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, azure_endpoint='https://corestackgenai.openai.azure.com', azure_ad_token=None, azure_ad_token_provider=None, validate_base_url=True)

In [58]:
index = VectorstoreIndexCreator(vectorstore_cls = DocArrayInMemorySearch, 
                                embedding = embeddings).from_loaders([loader])

In [62]:
chat = AzureChatOpenAI(temperature=0, azure_deployment = "gpt4", verbose=True)

In [63]:

qa = RetrievalQA.from_chain_type(
    llm=chat, 
    chain_type="stuff", 
    retriever=index.vectorstore.as_retriever(), 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

In [64]:
data[10]

Document(page_content="no: 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded hem.\n- Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg.\n\nImported.", metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 10})

In [65]:
data[11]

Document(page_content='no: 11\nname: Ultra-Lofty 850 Stretch Down Hooded Jacket\ndescription: This technical stretch down jacket from our DownTek collection is sure to keep you warm and comfortable with its full-stretch construction providing exceptional range of motion. With a slightly fitted style that falls at the hip and best with a midweight layer, this jacket is suitable for light activity up to 20° and moderate activity up to -30°. The soft and durable 100% polyester shell offers complete windproof protection and is insulated with warm, lofty goose down. Other features include welded baffles for a no-stitch construction and excellent stretch, an adjustable hood, an interior media port and mesh stash pocket and a hem drawcord. Machine wash and dry. Imported.', metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 11})

In [66]:
examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

## Agents

In [7]:
# from langchain.agents.agent_toolkits import 
from langchain.agents import load_tools
from langchain.agents import AgentExecutor
from langchain.agents.json_chat.base import create_json_chat_agent
# from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain import hub

In [8]:
chat = AzureChatOpenAI(temperature=0, azure_deployment = "gpt4", verbose=True)

In [9]:
tools = load_tools(["llm-math", "wikipedia"], llm = chat)

In [10]:
prompt = hub.pull("hwchase17/react-chat-json")

In [11]:
agent = create_json_chat_agent(tools=tools, llm=chat, prompt= prompt)
agent_executor = AgentExecutor(agent= agent, tools= tools, verbose=True)

In [12]:
agent_executor.invoke({"input":"What is the 25% of 300?"})



> Entering new AgentExecutor chain...
```json
{
    "action": "Calculator",
    "action_input": "25% of 300"
}
```Answer: 75.0```json
{
    "action": "Final Answer",
    "action_input": "The 25% of 300 is 75."
}
```

> Finished chain.


{'input': 'What is the 25% of 300?', 'output': 'The 25% of 300 is 75.'}

In [13]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
agent_executor.invoke({"input": question})



> Entering new AgentExecutor chain...
```json
{
    "action": "wikipedia",
    "action_input": "Tom M. Mitchell"
}
```

c:\Users\Imaran Asalam\AppData\Local\pypoetry\Cache\virtualenvs\ai-oM_CUHOU-py3.11\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\Imaran Asalam\AppData\Local\pypoetry\Cache\virtualenvs\ai-oM_CUHOU-py3.11\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Page: Tom M. Mitchell
Summary: Tom Michael Mitchell (born August 9, 1951) is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). He is a founder and former Chair of the Machine Learning Department at CMU. Mitchell is known for his contributions to the advancement of machine learning, artificial intelligence, and cognitive neuroscience and is the author of the textbook Machine Learning. He is a member of the United States National Academy of Engineering since 2010. He is also a Fellow of the American Academy of Arts and Sciences, the American Association for the Advancement of Science and a Fellow and past President of the Association for the Advancement of Artificial Intelligence. In October 2018, Mitchell was appointed as the Interim Dean of the School of Computer Science at Carnegie Mellon.

Page: Ensemble learning
Summary: In statistics and machine learning, ensemble methods use multiple learning algorithms to obtain better predi

{'input': 'Tom M. Mitchell is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU)what book did he write?',
 'output': "Tom M. Mitchell is the author of the textbook 'Machine Learning'."}